<a href="https://colab.research.google.com/github/satz2901/congenial-barnacle/blob/main/Iris_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
from __future__ import absolute_import,print_function,division,unicode_literals
import tensorflow as tf
import pandas as pd
import numpy as np
CSV_COLUMN_NAMES=['SepalLength','SepalWidth','PetalLength','PetalWidth','Species']
SPECIES=['Sentosa','Versicolor','Virginica']
train_path=tf.keras.utils.get_file("iris_training.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path=tf.keras.utils.get_file("iris_test.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")
train=pd.read_csv(train_path,names=CSV_COLUMN_NAMES,header=0)
test=pd.read_csv(test_path,names=CSV_COLUMN_NAMES,header=0)
train_y=train.pop('Species')
test_y=test.pop('Species')
def input_fn (features,labels,training=True,batch_size=256):
  dataset=tf.data.Dataset.from_tensor_slices((dict(features),labels))
  if training:
    dataset=dataset.shuffle(1000).repeat()
  return dataset.batch(batch_size)
my_feature_column=[]
for key in train.keys():
  my_feature_column.append(tf.feature_column.numeric_column(key=key))
classifier=tf.estimator.DNNClassifier(feature_columns=my_feature_column,hidden_units=[30,10],n_classes=3)
classifier.train(input_fn= lambda:input_fn(train,train_y,training=True),steps=5000)
eval_result=classifier.evaluate(input_fn=lambda: input_fn(test,test_y,training=False))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))
def input_fn(features,batch_size=256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)
features=['SepalLength','SepalWidth','PetalLength','PetalWidth']
predict={}
print("please type numeric value as prompted:")
for feature in features:
  valid=True
  while valid:
    val=input(feature+":")
    if not val.isdigit(): valid=False
  predict[feature]=[float(val)]
predictions=classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
  class_id=pred_dict['class_ids'][0]
  probability=pred_dict['probabilities'][class_id]
  print('prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id],100*probability))